In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/original-henrys-constant-dataset/Henry constants for COREMOFs.xlsx - Henry constants for COREMOFs.csv
/kaggle/input/enthalpy-and-henrys-constant-of-1709-mofs/Enthalpy and Henrys Constant of 1709 MOFs.xlsx - Sheet1.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import DBSCAN
from sklearn.model_selection import KFold,train_test_split, GridSearchCV, RandomizedSearchCV
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error


In [3]:
og_dataset1=pd.read_csv('/kaggle/input/original-henrys-constant-dataset/Henry constants for COREMOFs.xlsx - Henry constants for COREMOFs.csv')
og_dataset1

,MOF name,CSD Refcode,Database Acronym,Generic Identity,Calculation Type,Density,Gravimetric Surface Area,Volumetric Surface Area,Void Fraction,Pore Volume,...,Kh N2/Kh H2O,Kh N2O/Kh H2O,Kh O2/Kh H2O,Kh N2O/Kh N2,Kh O2/Kh N2,Kh O2/Kh N2O,LCD,PLD,SA,Void Fraction.1
0,ABAVIJ,ABAVIJ_clean,CoRE 2019,Real,Machine Learning,1.524930,0.000,0.00,0.3792,0.248667,...,1.354412e-01,2.981735e+01,4.323529e-01,220.149837,3.192182e+00,1.450004e-02,4.45543,2.49720,650.0860,0.38194
1,ABAYIO,ABAYIO_clean,CoRE 2019,Real,Machine Learning,0.949126,1505.310,1428.73,0.6636,0.699170,...,1.764526e-03,1.928414e-02,9.013832e-03,10.928793,5.108359e+00,4.674221e-01,11.39486,4.32260,3064.3400,0.66204
2,ABAYOU,ABAYOU_clean,CoRE 2019,Real,Machine Learning,0.979267,1494.400,1463.42,0.6638,0.677854,...,2.410000e-06,2.700000e-05,1.180000e-05,11.201201,4.894895e+00,4.369973e-01,11.27344,4.51080,3036.5600,0.66192
3,ABESUX,ABESUX_clean,CoRE 2019,Real,Machine Learning,3.742910,466.934,1747.69,0.5646,0.150845,...,4.270000e-12,8.570000e-08,5.100000e-12,20078.358210,1.194030e+00,5.950000e-05,5.87931,4.42585,637.7710,0.55732
4,ABETAE,ABETAE_clean,CoRE 2019,Real,Machine Learning,3.699280,492.461,1821.75,0.5720,0.154625,...,2.690000e-11,4.590000e-07,3.290000e-11,17054.153850,1.223077e+00,7.170000e-05,5.88295,4.42122,656.7440,0.56220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7659,ZUTBUN,ZUTBUN_clean,CoRE 2019,Real,Machine Learning,2.282800,454.670,1037.92,0.4626,0.202646,...,2.120330e-04,2.133714e-01,1.375182e-03,1006.314286,6.485714e+00,6.445019e-03,3.31800,2.48073,310.0820,0.36016
7660,ZUVTEP,ZUVTEP_clean,CoRE 2019,Real,Machine Learning,1.639520,0.000,0.00,0.5300,0.323266,...,1.902137e+04,1.416701e+07,3.260000e-05,744.794363,1.710000e-09,2.300000e-12,2.74525,2.44183,98.0872,0.31180
7661,ZUWXUM,ZUWXUM_clean,CoRE 2019,Real,Machine Learning,0.917254,2753.990,2526.11,0.7012,0.764456,...,3.704918e-01,3.901639e+00,9.590164e-01,10.530973,2.588496e+00,2.457983e-01,5.63288,5.50957,1897.5700,0.55592
7662,ZUXPOZ,ZUXPOZ_clean,CoRE 2019,Real,Machine Learning,2.001620,656.635,1314.33,0.5790,0.289266,...,1.440000e-05,5.770992e-02,6.280000e-06,4000.471976,4.351032e-01,1.087630e-04,6.77398,3.93575,2152.2200,0.56828


In [4]:
id=pd.DataFrame(og_dataset1['MOF name'])

In [5]:
phy_props=og_dataset1.iloc[:,5:12]
chem_props=og_dataset1.iloc[:,13:29]
df1=phy_props.join(chem_props)
df1=id.join(df1)
print(df1)

     MOF name   Density  Gravimetric Surface Area  Volumetric Surface Area  \
0      ABAVIJ  1.524930                     0.000                     0.00   
1      ABAYIO  0.949126                  1505.310                  1428.73   
2      ABAYOU  0.979267                  1494.400                  1463.42   
3      ABESUX  3.742910                   466.934                  1747.69   
4      ABETAE  3.699280                   492.461                  1821.75   
...       ...       ...                       ...                      ...   
7659   ZUTBUN  2.282800                   454.670                  1037.92   
7660   ZUVTEP  1.639520                     0.000                     0.00   
7661   ZUWXUM  0.917254                  2753.990                  2526.11   
7662   ZUXPOZ  2.001620                   656.635                  1314.33   
7663   ZUZZEB  0.903178                  2232.360                  2016.22   

      Void Fraction  Pore Volume  Largest Cavity Diameter  \
0 

In [6]:
df2=pd.read_csv('/kaggle/input/enthalpy-and-henrys-constant-of-1709-mofs/Enthalpy and Henrys Constant of 1709 MOFs.xlsx - Sheet1.csv')
list=df2.columns.tolist()
list[0]='MOF name'
df2.columns=list
df2

,MOF name,Kh of NH3,Kh of N2,Kh of H2,enthalpy of NH3,enthalpy of N2,enthalpy of H2
0,ABEXIQ,2.050000e+14,0.000004,0.000005,-49.64259,-16.87899,-26.71471
1,ABEXOW,1.730000e+11,0.000003,0.000005,-49.35955,-13.0402,-27.48331
2,ABEXUC,6.250000e+12,0.000003,0.000005,-54.52562,-27.57187,-24.77261
3,ABEYAJ,7.370000e+13,0.000003,0.000005,-57.70495,-23.79712,-30.33349
4,ABEYIR,3.890000e+13,0.000003,0.000005,-52.25057,-17.9982,-24.70991
...,...,...,...,...,...,...,...
1704,ZnBDC,3.290720e-01,0.000007,0.000009,-36.07089,-18.22681,-19.54107
1705,ZnBDCdabco,2.880000e-05,0.000003,0.000005,-21.6735,-11.49415,-12.30477
1706,ZnHBDC,2.035850e-01,0.000003,0.000005,-37.17837,-12.26908,-12.91726
1707,ZnMOF-74,2.370000e-04,0.000003,0.000004,-34.32558,-11.80274,-13.30799


In [7]:
df=pd.merge(df1,df2,on='MOF name',how='inner')
id=df['MOF name']
df=df.drop(['MOF name'],axis='columns')
print(df)

       Density  Gravimetric Surface Area  Volumetric Surface Area  \
0     1.787600                   534.065                  954.695   
1     1.812430                   523.222                  948.305   
2     1.848130                   490.198                  905.952   
3     1.969620                   446.707                  879.845   
4     1.346610                   418.836                  564.007   
...        ...                       ...                      ...   
1115  0.782965                  3272.960                 2562.610   
1116  0.969805                  2246.270                 2178.440   
1117  0.952981                  1420.370                 1353.580   
1118  1.288920                   900.270                 1160.380   
1119  1.306180                   983.341                 1284.420   

      Void Fraction  Pore Volume  Largest Cavity Diameter  \
0            0.4776     0.267174                  4.80529   
1            0.5026     0.277307                 

In [8]:
df.describe()

,Density,Gravimetric Surface Area,Volumetric Surface Area,Void Fraction,Pore Volume,Largest Cavity Diameter,Pore Limiting Diameter,H,C,N,...,Zr,total degree of unsaturation,metalic percentage,oxygetn-to-metal ratio,electronegtive-to-total ratio,weighted electronegativity per atom,nitrogen to oxygen,Kh of NH3,Kh of N2,Kh of H2
count,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,...,1120.0,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1.120000e+03,1.120000e+03,1.120000e+03
mean,1.286546,1284.528779,1424.635697,0.562605,0.490914,6.579781,5.132020,39.929464,59.862500,9.441071,...,0.0,38.167857,16.958573,4.008484,0.262840,0.877350,0.373327,3.034606e+21,5.242240e-06,7.375152e-06
std,0.377224,812.893509,569.752742,0.090101,0.225171,2.010815,1.563131,30.228907,36.695019,13.389808,...,0.0,25.635479,25.356144,2.476486,0.108316,0.368534,1.039957,1.004033e+23,5.657133e-06,7.409684e-06
min,0.407232,0.000000,0.000000,0.345400,0.111729,4.001690,2.885900,0.000000,4.000000,0.000000,...,0.0,-18.500000,2.083333,0.000000,0.000000,0.000000,0.000000,2.870000e-06,1.410000e-07,4.200000e-07
25%,1.036573,671.502250,957.723000,0.495750,0.337761,5.185873,4.016635,18.000000,32.000000,0.000000,...,0.0,19.000000,6.956696,3.000000,0.181409,0.598261,0.000000,1.260000e-04,2.540000e-06,3.837500e-06
50%,1.223445,1102.295000,1372.895000,0.559800,0.447820,5.998275,4.645915,32.000000,52.000000,4.000000,...,0.0,33.000000,11.111111,4.000000,0.237456,0.809412,0.000000,5.697000e-03,4.030000e-06,5.990000e-06
75%,1.492340,1676.752500,1831.407500,0.623850,0.583526,7.280055,5.631045,56.000000,80.000000,16.000000,...,0.0,53.500000,16.666667,6.000000,0.324324,1.086316,0.370066,1.049335e+02,6.130000e-06,9.102500e-06
max,3.913040,4944.370000,3226.030000,0.866600,2.128030,17.925160,17.513950,216.000000,208.000000,96.000000,...,0.0,143.000000,437.500000,19.000000,0.666667,2.293333,16.000000,3.360000e+24,9.450000e-05,1.730000e-04


In [9]:
df=df.map(lambda x:0 if isinstance(x,str) else x)
df

,Density,Gravimetric Surface Area,Volumetric Surface Area,Void Fraction,Pore Volume,Largest Cavity Diameter,Pore Limiting Diameter,H,C,N,...,oxygetn-to-metal ratio,electronegtive-to-total ratio,weighted electronegativity per atom,nitrogen to oxygen,Kh of NH3,Kh of N2,Kh of H2,enthalpy of NH3,enthalpy of N2,enthalpy of H2
0,1.787600,534.065,954.695,0.4776,0.267174,4.80529,4.06893,16,56,16,...,6.000000,0.444444,1.484444,0.333333,2.050000e+14,0.000004,0.000005,0,0,0
1,1.812430,523.222,948.305,0.5026,0.277307,4.77270,4.04921,16,56,16,...,6.000000,0.444444,1.484444,0.333333,1.730000e+11,0.000003,0.000005,0,0,0
2,1.848130,490.198,905.952,0.4864,0.263185,4.74458,4.01680,16,56,16,...,6.000000,0.444444,1.484444,0.333333,6.250000e+12,0.000003,0.000005,0,0,0
3,1.969620,446.707,879.845,0.4640,0.235578,4.69665,3.92183,16,56,16,...,6.000000,0.444444,1.484444,0.333333,3.890000e+13,0.000003,0.000005,0,0,0
4,1.346610,418.836,564.007,0.4458,0.331055,4.33176,3.82431,112,160,40,...,2.666667,0.202247,0.650787,1.250000,5.777000e-03,0.000002,0.000005,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,0.782965,3272.960,2562.610,0.6992,0.893015,7.07783,5.58584,30,48,2,...,4.000000,0.180000,0.611200,0.125000,7.920000e-05,0.000004,0.000007,0,0,0
1116,0.969805,2246.270,2178.440,0.6456,0.665701,5.81598,3.95730,30,48,2,...,4.000000,0.180000,0.611200,0.125000,3.030000e-04,0.000005,0.000011,0,0,0
1117,0.952981,1420.370,1353.580,0.5664,0.594346,4.97187,3.80620,64,96,8,...,4.000000,0.127660,0.422128,0.500000,5.469600e-02,0.000004,0.000010,0,0,0
1118,1.288920,900.270,1160.380,0.5204,0.403748,5.90990,4.91539,64,90,8,...,0.000000,0.047059,0.143059,0.000000,1.830000e-04,0.000008,0.000011,0,0,0


In [10]:
std_scaler=StandardScaler()
df=pd.DataFrame(std_scaler.fit_transform(df),columns=df.columns)
df

,Density,Gravimetric Surface Area,Volumetric Surface Area,Void Fraction,Pore Volume,Largest Cavity Diameter,Pore Limiting Diameter,H,C,N,...,oxygetn-to-metal ratio,electronegtive-to-total ratio,weighted electronegativity per atom,nitrogen to oxygen,Kh of NH3,Kh of N2,Kh of H2,enthalpy of NH3,enthalpy of N2,enthalpy of H2
0,1.328859,-0.923613,-0.825184,-0.943868,-0.994087,-0.882868,-0.680406,-0.791962,-0.105307,0.490064,...,0.804530,1.677366,1.648058,-0.038474,-0.030238,-0.293962,-0.324741,0.0,0.0,0.0
1,1.394712,-0.936958,-0.836404,-0.666276,-0.949066,-0.899082,-0.693028,-0.791962,-0.105307,0.490064,...,0.804530,1.677366,1.648058,-0.038474,-0.030238,-0.373543,-0.332842,0.0,0.0,0.0
2,1.489393,-0.977601,-0.910773,-0.846156,-1.011811,-0.913073,-0.713771,-0.791962,-0.105307,0.490064,...,0.804530,1.677366,1.648058,-0.038474,-0.030238,-0.332868,-0.322040,0.0,0.0,0.0
3,1.811600,-1.031127,-0.956615,-1.094877,-1.134470,-0.936920,-0.774554,-0.791962,-0.105307,0.490064,...,0.804530,1.677366,1.648058,-0.038474,-0.030238,-0.423060,-0.334192,0.0,0.0,0.0
4,0.159297,-1.065428,-1.511205,-1.296964,-0.710261,-1.118465,-0.836970,2.385225,2.730132,2.283272,...,-0.542065,-0.559655,-0.615045,0.843367,-0.030238,-0.610518,-0.315289,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,-1.335562,2.447208,1.998205,1.516701,1.786559,0.247796,0.290457,-0.328623,-0.323417,-0.555975,...,-0.003427,-0.765138,-0.722509,-0.238893,-0.030238,-0.200233,-0.049302,0.0,0.0,0.0
1116,-0.840038,1.183637,1.323629,0.921545,0.776590,-0.380016,-0.751853,-0.328623,-0.323417,-0.555975,...,-0.003427,-0.765138,-0.722509,-0.238893,-0.030238,-0.033997,0.543430,0.0,0.0,0.0
1117,-0.884657,0.167183,-0.124769,0.042136,0.459556,-0.799989,-0.848561,0.796631,0.985247,-0.107673,...,-0.003427,-1.248572,-1.235777,0.121860,-0.030238,-0.263898,0.335501,0.0,0.0,0.0
1118,0.006296,-0.472916,-0.464015,-0.468632,-0.387282,-0.333288,-0.138649,0.796631,0.821664,-0.107673,...,-1.619341,-1.993030,-1.993355,-0.359144,-0.030238,0.422268,0.502925,0.0,0.0,0.0


In [11]:
dbscan = DBSCAN(eps=3, min_samples=25)
clusters = dbscan.fit_predict(df)
df = df[clusters != -1]
df

,Density,Gravimetric Surface Area,Volumetric Surface Area,Void Fraction,Pore Volume,Largest Cavity Diameter,Pore Limiting Diameter,H,C,N,...,oxygetn-to-metal ratio,electronegtive-to-total ratio,weighted electronegativity per atom,nitrogen to oxygen,Kh of NH3,Kh of N2,Kh of H2,enthalpy of NH3,enthalpy of N2,enthalpy of H2
0,1.328859,-0.923613,-0.825184,-0.943868,-0.994087,-0.882868,-0.680406,-0.791962,-0.105307,0.490064,...,0.804530,1.677366,1.648058,-0.038474,-0.030238,-0.293962,-0.324741,0.0,0.0,0.0
1,1.394712,-0.936958,-0.836404,-0.666276,-0.949066,-0.899082,-0.693028,-0.791962,-0.105307,0.490064,...,0.804530,1.677366,1.648058,-0.038474,-0.030238,-0.373543,-0.332842,0.0,0.0,0.0
2,1.489393,-0.977601,-0.910773,-0.846156,-1.011811,-0.913073,-0.713771,-0.791962,-0.105307,0.490064,...,0.804530,1.677366,1.648058,-0.038474,-0.030238,-0.332868,-0.322040,0.0,0.0,0.0
3,1.811600,-1.031127,-0.956615,-1.094877,-1.134470,-0.936920,-0.774554,-0.791962,-0.105307,0.490064,...,0.804530,1.677366,1.648058,-0.038474,-0.030238,-0.423060,-0.334192,0.0,0.0,0.0
5,-0.340045,0.273407,0.562942,0.648396,0.200846,-0.746345,-0.705073,0.531866,0.985247,-0.705409,...,-0.205416,-0.118593,-0.047100,-0.359144,-0.030238,0.728213,0.529928,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,-1.335562,2.447208,1.998205,1.516701,1.786559,0.247796,0.290457,-0.328623,-0.323417,-0.555975,...,-0.003427,-0.765138,-0.722509,-0.238893,-0.030238,-0.200233,-0.049302,0.0,0.0,0.0
1116,-0.840038,1.183637,1.323629,0.921545,0.776590,-0.380016,-0.751853,-0.328623,-0.323417,-0.555975,...,-0.003427,-0.765138,-0.722509,-0.238893,-0.030238,-0.033997,0.543430,0.0,0.0,0.0
1117,-0.884657,0.167183,-0.124769,0.042136,0.459556,-0.799989,-0.848561,0.796631,0.985247,-0.107673,...,-0.003427,-1.248572,-1.235777,0.121860,-0.030238,-0.263898,0.335501,0.0,0.0,0.0
1118,0.006296,-0.472916,-0.464015,-0.468632,-0.387282,-0.333288,-0.138649,0.796631,0.821664,-0.107673,...,-1.619341,-1.993030,-1.993355,-0.359144,-0.030238,0.422268,0.502925,0.0,0.0,0.0


In [12]:
X=df.iloc[:,1:24]
Y=df.iloc[:,24:30]

In [13]:
def getscore(model,X1,X2,Y1,Y2):
    model.fit(X1,Y1)
    return model.score(X2,Y2)

In [14]:
knn=KNeighborsRegressor(n_neighbors=5)
score_avg=0
for i in range(0,100):
    X1,X2,Y1,Y2=train_test_split(X,Y,test_size=0.3,random_state=i)
    knn.fit(X1,Y1)
    score_avg+=knn.score(X2,Y2)

score_avg/=100
    
print(score_avg)

0.7593606436903486


In [15]:
dt=DecisionTreeRegressor()
score_avg=0
for i in range(0,100):
    X1,X2,Y1,Y2=train_test_split(X,Y,test_size=0.3,random_state=i)
    dt.fit(X1,Y1)
    score_avg+=dt.score(X2,Y2)

score_avg/=100
    
print(score_avg)

0.6203051384560108


In [16]:
rf=RandomForestRegressor()
score_avg=0
for i in range(0,100):
    X1,X2,Y1,Y2=train_test_split(X,Y,test_size=0.3,random_state=i)
    rf.fit(X1,Y1)
    score_avg+=rf.score(X2,Y2)

score_avg/=100
    
print(score_avg)

0.8114651840034395


In [17]:
score_avg=0
for i in range(0,50):
    X1,X2,Y1,Y2=train_test_split(X,Y,test_size=0.3,random_state=i)
    model=lgb.LGBMRegressor(metric= 'r2',boosting_type='dart',learning_rate=0.27,num_leaves=32,max_depth=10,n_estimators=39,verbose=-1)
    reg=MultiOutputRegressor(model)
    reg.fit(X1,Y1)
    Y_pred=reg.predict(X2)
    score_avg+=r2_score(Y2,Y_pred)



score_avg/=50
print(score_avg)

0.8311254928843941


In [18]:
X1,X2,Y1,Y2=train_test_split(X,Y,test_size=0.3,random_state=0)
d1=xgb.DMatrix(X1,label=Y1)
d2=xgb.DMatrix(X2,label=Y2)
clf=xgb.train(params={
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',  # R-squared as the evaluation metric
    'learning_rate': 0.1,
    'max_depth': 5,
    'alpha': 10
},num_boost_round=100,dtrain=d1)

Y_pred=clf.predict(d2)

Y_avg=np.mean(Y2,axis=0)
tss=np.sum((Y2-Y_avg)**2)
rss=np.sum((Y2-Y_pred)**2)
r2=1-(rss/tss)
r2

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Kh of N2           0.524051
Kh of H2           0.529055
enthalpy of NH3        -inf
enthalpy of N2         -inf
enthalpy of H2         -inf
dtype: float64

X1,X2,Y1,Y2=train_test_split(X,Y,test_size=0.3,random_state=i)
parameters={'learning_rate':[i/100 for i in range(0,100)]
            'n_jobs':[i for i in range(0,100)]
            'boosting_type':['gbdt','dart']
            'max_depth':[i for i in range(-1,50)]
    
}
reg_CV=GridSearchCV(MultiOutputRegressor(lgb.LGBMRegressor()),parameters,cv=5)
reg_CV.fit(X1,Y1)
reg_CV.best_params_

In [19]:
X1,X2,Y1,Y2=train_test_split(X,Y,test_size=0.3,random_state=0)
parameters={'estimator__learning_rate':[i/100 for i in range(0,50)],
            'estimator__n_estimators':[i for i in range(0,50)],
            'estimator__boosting_type':['gbdt','dart'], 
            'estimator__max_depth':[-1,10,20],
            'estimator__num_leaves':[i for i in range(1,50)],
}
model=MultiOutputRegressor(lgb.LGBMRegressor())
reg_CV=RandomizedSearchCV(model,parameters,cv=5)
reg_CV.fit(X1,Y1)
print(reg_CV.best_params_)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000082 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1529
[LightGBM] [Info] Number of data points in the train set: 494, number of used features: 18
[LightGBM] [Info] Start training from score -0.022339
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000082 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1529
[LightGBM] [Info] Number of data points in the train set: 494, number of used features: 18
[LightGBM] [Info] Start training from score 0.015166
[LightGBM] [Warning] Found whit

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/multioutput.py", line 216, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "/opt/conda/lib/python3.10/site-packages/sklearn/utils/parallel.py", line 63, in __call__
    return super().__call__(iterable_with_config)
  File "/opt/conda/lib/pytho